In [1]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
from keras.datasets import mnist
from keras.layers import Dense, Flatten, Reshape
from keras.layers.advanced_activations import LeakyReLU
from keras.models import Sequential
from keras.optimizers import Adam


**Model input dimensions**

In [2]:
img_rows=28
img_cols=28
channels=1

img_shape=(img_rows,img_cols,channels)

#Noise Vector
z_dim=100

**Generator implementation**

In [3]:
def build_generator(img_shape,z_dim):
  model=Sequential()
  model.add(Dense(128,input_dim=(z_dim)))
  model.add(LeakyReLU(alpha=0.01))
  model.add(Dense(28*28*1,activation='tanh'))
  model.add(Reshape(img_shape))

  return model

**Discriminator implementation**

In [4]:
def build_discriminator(img_shape):
  model=Sequential()
  model.add(Flatten(input_shape=img_shape))
  model.add(Dense(128))
  model.add(LeakyReLU(alpha=0.01))
  model.add(Dense(1,activation='sigmoid'))

  return model

**Building GAN**

In [6]:
def build_gan(generator,discriminator):
  model=Sequential()
  model.add(generator)
  model.add(discriminator)

  return model

discriminator=build_discriminator(img_shape)
discriminator.compile(loss='binary_crossentropy', optimizer=Adam(),metrics=['accuracy'])

generator=build_generator(img_shape,z_dim)

discriminator.trainable=False

gan=build_gan(generator,discriminator)

gan.compile(loss='binary_crossentropy',optimizer=Adam())






**GAN Training**

In [9]:
losses=[]
accuracies=[]
iteration_checkpoints=[]

def train(iterations,batch_size,sample_interval):
  (X_train,_) , (_,_)=mnist.load_data()
  X_train=X_train/127.5 - 1.0
  X_train=np.expand_dims(X_train, axis=3)

  real=np.ones((batch_size,1))
  fake=np.zeros((batch_size,1))

  for iteration in range(iterations):
    
    idx=np.random.randint(0,X_train.shape[0],batch_size)
    imgs=X_train[idx]

    z=np.random.normal(0,1,(batch_size,100))
    gen_imgs=generator.predict(z)
    
    d_loss_real=discriminator.train_on_batch(imgs,real)
    d_loss_fake=discriminator.train_on_batch(gen_imgs,fake)
    d_loss, accuracy=0.5*np.add(d_loss_real,d_loss_fake)

    z=np.random.normal(0,1,(batch_size,100))
    gen_imgs=generator.predict(z)

    g_loss=gan.train_on_batch(z,real)

    if (iteration+1) % sample_interval == 0:
      losses.append((d_loss,g_loss))
      accuracies.append(100.0*accuracy)
      iteration_checkpoints.append(iteration+1)

      print("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (iteration + 1, d_loss, 100.0 * accuracy, g_loss))



**Run the model**

In [11]:
iterations=10000
batch_size=128
sample_interval=50

train(iterations,batch_size,sample_interval)


50 [D loss: 0.006204, acc.: 100.00%] [G loss: 7.192741]
100 [D loss: 0.027969, acc.: 100.00%] [G loss: 3.854657]
150 [D loss: 0.037176, acc.: 100.00%] [G loss: 2.763245]
200 [D loss: 0.025855, acc.: 100.00%] [G loss: 3.135652]
250 [D loss: 0.012532, acc.: 100.00%] [G loss: 3.858117]
300 [D loss: 0.006857, acc.: 100.00%] [G loss: 4.566443]
350 [D loss: 0.005946, acc.: 100.00%] [G loss: 4.874016]
400 [D loss: 0.006660, acc.: 100.00%] [G loss: 5.017665]
450 [D loss: 0.005746, acc.: 100.00%] [G loss: 4.876806]
500 [D loss: 0.003495, acc.: 100.00%] [G loss: 5.234996]
550 [D loss: 0.004231, acc.: 100.00%] [G loss: 5.560380]
600 [D loss: 0.004416, acc.: 100.00%] [G loss: 5.812941]
650 [D loss: 0.004244, acc.: 100.00%] [G loss: 6.387346]
700 [D loss: 0.062967, acc.: 99.22%] [G loss: 3.219258]
750 [D loss: 0.018735, acc.: 100.00%] [G loss: 4.754429]
800 [D loss: 0.104915, acc.: 98.05%] [G loss: 3.020444]
850 [D loss: 0.066195, acc.: 98.44%] [G loss: 4.610833]
900 [D loss: 0.086025, acc.: 97.66%